In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=3bf40ba4b667447142b829660b000003bcbffb62fd88bd251a1c7918d3b8a283
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("classification").getOrCreate()

In [ ]:
hrdata=spark.read.csv("/content/drive/MyDrive/HR Analytics/train_LZdllcl.csv",
                      inferSchema=True,header=True)

In [ ]:
hrdata.groupBy('education').count().show()

+----------------+-----+
|       education|count|
+----------------+-----+
| Below Secondary|  805|
|Master's & above|14925|
|      Bachelor's|39078|
+----------------+-----+



In [ ]:
hrdata=hrdata.na.fill(value="Bachelor's",subset=['education'])

In [ ]:
hrdata.groupBy('previous_year_rating').count().show()

+--------------------+-----+
|previous_year_rating|count|
+--------------------+-----+
|                   1| 6223|
|                   3|22742|
|                   5|11741|
|                   4| 9877|
|                   2| 4225|
+--------------------+-----+



In [ ]:
hrdata=hrdata.na.fill(value=3,subset=['previous_year_rating'])

In [ ]:
hrdata.columns

['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met >80%',
 'awards_won?',
 'avg_training_score',
 'is_promoted']

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [ ]:
indexer=[StringIndexer(inputCol=col,outputCol=col+"index").fit(hrdata)
for col in list(set(hrdata.columns)-set(['no_of_trainings','age',
                                         'length_of_service',
                                         'avg_training_score','employee_id']))]

In [ ]:
pipeline=Pipeline(stages=indexer)

In [ ]:
hrdatadf=pipeline.fit(hrdata).transform(hrdata)

In [ ]:
hrdatadf.columns

['employee_id',
 'department',
 'no_of_trainings',
 'age',
 'length_of_service',
 'avg_training_score',
 'genderindex',
 'is_promotedindex',
 'KPIs_met >80%index',
 'awards_won?index',
 'educationindex',
 'recruitment_channelindex',
 'departmentindex',
 'regionindex',
 'previous_year_ratingindex',
 'features',
 'label']

In [ ]:
columnstodrop=['employee_id''department','region','education','gender',
               'recruitment_channel','previous_year_rating','KPIs_met >80%',
               'awards_won?','is_promoted']

In [ ]:
hrdatadf=hrdatadf.drop(*columnstodrop)

In [ ]:
from pyspark.ml.feature import RFormula

In [ ]:
formula=RFormula(formula="is_promotedindex~.",featuresCol='features',
                 labelCol='label')

In [ ]:
hrdatadf=formula.fit(hrdatadf).transform(hrdatadf)

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

In [ ]:
logit=LogisticRegression()

In [ ]:
logitmodel=logit.fit(hrdatadf)

In [ ]:
logitmodel.summary.accuracy

0.9325098525762663

In [ ]:
logitmodel.summary.areaUnderROC

0.8683283464397373

In [ ]:
accuracy=MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
auc=BinaryClassificationEvaluator()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
tree=DecisionTreeClassifier(maxBins=35)

In [ ]:
treemodel=tree.fit(hrdatadf)

In [ ]:
treepredict=treemodel.transform(hrdatadf)

In [ ]:
accuracy.evaluate(treepredict)

0.9261786600496278

In [ ]:
auc.evaluate(treepredict)

0.5785816850778404

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
RF=RandomForestClassifier(maxBins=35)

In [ ]:
RFmodel=RF.fit(hrdatadf)

In [ ]:
RFpredict=RFmodel.transform(hrdatadf)

In [ ]:
accuracy.evaluate(RFpredict)

0.9237520070062765

In [ ]:
auc.evaluate(RFpredict)

0.8557354168397036

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
gbm=GBTClassifier(maxBins=35)

In [ ]:
gbmmodel=gbm.fit(hrdatadf)

In [ ]:
gbmpredict=gbmmodel.transform(hrdatadf)

In [ ]:
accuracy.evaluate(gbmpredict)

0.9419245365640052

In [ ]:
auc.evaluate(gbmpredict)

0.9169767944528242

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
nn=MultilayerPerceptronClassifier(layers=[11,100,2])
# 11 - input , hidden layer 1 with 100 neurons and output layer 2

In [ ]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
nnmodel=nn.fit(hrdatadf)

In [ ]:
nnpredict=nnmodel.transform(hrdatadf)